In [28]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from modules.bag_reader.bag_reader import READER
from modules.bag_processor.bag_processor import DepthProcessor


bag_read = READER('/home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-27-12-42-56.bag')
bag_read.read()
bag_read.depth_df

dp = DepthProcessor()
feat_df = dp.get_features_df(bag_read.depth_df)

print(feat_df["std"][4])

[INFO]  Data folder /home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-27-12-42-56 already exists. Not creating.


IndexError: list assignment index out of range

[INFO]  Data folder /home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-27-12-42-56 already exists. Not creating.


,Unnamed: 0,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,height,width,encoding,is_bigendian,step,frame_path,np_path
0,0,1.666864e+09,323,1666863775,837964519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
1,1,1.666864e+09,327,1666863776,371473519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
2,2,1.666864e+09,333,1666863776,971345519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
3,3,1.666864e+09,336,1666863777,304747519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
4,4,1.666864e+09,338,1666863777,571358519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,117,1.666864e+09,700,1666863815,706017519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
118,118,1.666864e+09,701,1666863815,839458519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
119,119,1.666864e+09,707,1666863816,439334519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...
120,120,1.666864e+09,708,1666863816,572845519,zedm_left_camera_optical_frame,360,640,16UC1,0,1280,/home/nimibot/catkin_ws/src/ros_env_prediction...,/home/nimibot/catkin_ws/src/ros_env_prediction...


IndexError: list assignment index out of range